通过[alarm](alarm.ipynb)给 python subprocess模块增加超时机制。

In [ ]:
# -*- coding: utf-8 -*-

r"""subprocess with timeout mechanism.
Since the subprocess module in Python's standard library can't handle subprocess
block permanently, we write this module to enhance subprocess module with a timeout
mechanism. User can specify a timeout value when invoking the run function and the
run function will return a result with exit status -9 if timeout happens.
"""

__all__ = ['run']

from os import kill
from signal import alarm, signal, SIGALRM, SIGKILL
from subprocess import PIPE, Popen


def run(args, cwd=None, shell=False, kill_tree=True, timeout=-1, env=None):
    """
    Run a command with a timeout after which it will be forcibly killed.
    """

    class Alarm(Exception):
        pass

    def alarm_handler(signum, frame):
        raise Alarm

    p = Popen(args, shell=shell, cwd=cwd, stdout=PIPE, stderr=PIPE, env=env)

    # setup alarm handler for timeout
    if timeout != -1:
        signal(SIGALRM, alarm_handler)
        alarm(timeout)

    try:
        out, err = p.communicate()
        # enter alarm handler to cancel previously set alarm
        if timeout != -1:
            alarm(0)
    except Alarm:
        pids = [p.pid]
        if kill_tree:
            pids.extend(get_child_process(p.pid))
        for pid in pids:
            # process might have died before getting to this line so wrap to avoid OSError
            try:
                kill(pid, SIGKILL)
            except OSError:
                pass
        # return in alarm
        return -9, "", ""

    return p.returncode, out, err


def get_child_process(pid):
    p = Popen("ps --no-headers -o pid --ppid {}".format(pid), shell=True, stdout=PIPE, stderr=PIPE)
    out = p.communicate()[0]
    return [int(p) for p in out.split()]
